In [1]:
import logging  # Để ghi log
import sys, os  # Để tương tác với hệ thống
import argparse  # Để xử lý tham số dòng lệnh
import random  # Để tạo số ngẫu nhiên

import pandas as pd  # Để xử lý dữ liệu dạng bảng
import torch  # Framework deep learning
from vsc.baseline.model_factory.utils import build_dataset  # Hàm tạo dataset
import torch.nn.functional as F  # Các hàm activation và loss
# from video.model import MS  # Mô hình MS 
import torch.distributed as dist  # Để huấn luyện phân tán
from torch.nn.parallel import DistributedDataParallel as DDP  # Wrapper cho mô hình phân tán
from mmcv import Config  # Để đọc file cấu hình
import numpy as np  # Để tính toán số học
from torch.utils.data import DataLoader  # Để tải dữ liệu
from transformers import AdamW, get_linear_schedule_with_warmup  # Optimizer và learning rate scheduler
from sklearn.metrics import average_precision_score  # Để tính điểm AP
import warnings
from transformers import BertTokenizer, BertModel, AutoConfig, AutoModel

d:\Miniconda\envs\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Miniconda\envs\myenv\Lib\site-packages\mmcv\__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
d:\Miniconda\envs\myenv\Lib\site-packages\albumentations\__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.20 (you have 1.4.17). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
class Registry:
    """A registry to map strings to classes.

    Registered object could be built from registry.
    Example:
        >>> MODELS = Registry('models')
        >>> @MODELS.register_module()
        >>> class ResNet:
        >>>     pass
        >>> resnet = MODELS.build(dict(type='ResNet'))

    Please refer to
    https://mmcv.readthedocs.io/en/latest/understand_mmcv/registry.html for
    advanced useage.

    Args:
        name (str): Registry name.
        build_func(func, optional): Build function to construct instance from
            Registry, func:`build_from_cfg` is used if neither ``parent`` or
            ``build_func`` is specified. If ``parent`` is specified and
            ``build_func`` is not given,  ``build_func`` will be inherited
            from ``parent``. Default: None.
        parent (Registry, optional): Parent registry. The class registered in
            children registry could be built from parent. Default: None.
        scope (str, optional): The scope of registry. It is the key to search
            for children registry. If not specified, scope will be the name of
            the package where class is defined, e.g. mmdet, mmcls, mmseg.
            Default: None.
    """

    def __init__(self, name, build_func=None, parent=None, scope=None):
        self._name = name
        self._module_dict = dict()
        self._children = dict()
        self._scope = self.infer_scope() if scope is None else scope

        # self.build_func will be set with the following priority:
        # 1. build_func
        # 2. parent.build_func
        # 3. build_from_cfg
        if build_func is None:
            if parent is not None:
                self.build_func = parent.build_func
            else:
                self.build_func = build_from_cfg
        else:
            self.build_func = build_func
        if parent is not None:
            assert isinstance(parent, Registry)
            parent._add_children(self)
            self.parent = parent
        else:
            self.parent = None

    def __len__(self):
        return len(self._module_dict)

    def __contains__(self, key):
        return self.get(key) is not None

    def __repr__(self):
        format_str = self.__class__.__name__ + \
                     f'(name={self._name}, ' \
                     f'items={self._module_dict})'
        return format_str

    @staticmethod
    def infer_scope():
        """Infer the scope of registry.

        The name of the package where registry is defined will be returned.

        Example:
            # in mmdet/models/backbone/resnet.py
            >>> MODELS = Registry('models')
            >>> @MODELS.register_module()
            >>> class ResNet:
            >>>     pass
            The scope of ``ResNet`` will be ``mmdet``.


        Returns:
            scope (str): The inferred scope name.
        """
        # inspect.stack() trace where this function is called, the index-2
        # indicates the frame where `infer_scope()` is called
        filename = inspect.getmodule(inspect.stack()[2][0]).__name__
        split_filename = filename.split('.')
        return split_filename[0]

    @staticmethod
    def split_scope_key(key):
        """Split scope and key.

        The first scope will be split from key.

        Examples:
            >>> Registry.split_scope_key('mmdet.ResNet')
            'mmdet', 'ResNet'
            >>> Registry.split_scope_key('ResNet')
            None, 'ResNet'

        Return:
            scope (str, None): The first scope.
            key (str): The remaining key.
        """
        split_index = key.find('.')
        if split_index != -1:
            return key[:split_index], key[split_index + 1:]
        else:
            return None, key

    @property
    def name(self):
        return self._name

    @property
    def scope(self):
        return self._scope

    @property
    def module_dict(self):
        return self._module_dict

    @property
    def children(self):
        return self._children

    def get(self, key):
        """Get the registry record.

        Args:
            key (str): The class name in string format.

        Returns:
            class: The corresponding class.
        """
        scope, real_key = self.split_scope_key(key)
        if scope is None or scope == self._scope:
            # get from self
            if real_key in self._module_dict:
                return self._module_dict[real_key]
        else:
            # get from self._children
            if scope in self._children:
                return self._children[scope].get(real_key)
            else:
                # goto root
                parent = self.parent
                while parent.parent is not None:
                    parent = parent.parent
                return parent.get(key)

    def build(self, *args, **kwargs):
        return self.build_func(*args, **kwargs, registry=self)

    def _add_children(self, registry):
        """Add children for a registry.

        The ``registry`` will be added as children based on its scope.
        The parent registry could build objects from children registry.

        Example:
            >>> models = Registry('models')
            >>> mmdet_models = Registry('models', parent=models)
            >>> @mmdet_models.register_module()
            >>> class ResNet:
            >>>     pass
            >>> resnet = models.build(dict(type='mmdet.ResNet'))
        """

        assert isinstance(registry, Registry)
        assert registry.scope is not None
        assert registry.scope not in self.children, \
            f'scope {registry.scope} exists in {self.name} registry'
        self.children[registry.scope] = registry

    def _register_module(self, module_class, module_name=None, force=False):
        if not inspect.isclass(module_class):
            raise TypeError('module must be a class, '
                            f'but got {type(module_class)}')

        if module_name is None:
            module_name = module_class.__name__
        if isinstance(module_name, str):
            module_name = [module_name]
        for name in module_name:
            if not force and name in self._module_dict:
                raise KeyError(f'{name} is already registered '
                               f'in {self.name}')
            self._module_dict[name] = module_class

    def deprecated_register_module(self, cls=None, force=False):
        warnings.warn(
            'The old API of register_module(module, force=False) '
            'is deprecated and will be removed, please use the new API '
            'register_module(name=None, force=False, module=None) instead.')
        if cls is None:
            return partial(self.deprecated_register_module, force=force)
        self._register_module(cls, force=force)
        return cls

    def register_module(self, name=None, force=False, module=None):
        """Register a module.

        A record will be added to `self._module_dict`, whose key is the class
        name or the specified name, and value is the class itself.
        It can be used as a decorator or a normal function.

        Example:
            >>> backbones = Registry('backbone')
            >>> @backbones.register_module()
            >>> class ResNet:
            >>>     pass

            >>> backbones = Registry('backbone')
            >>> @backbones.register_module(name='mnet')
            >>> class MobileNet:
            >>>     pass

            >>> backbones = Registry('backbone')
            >>> class ResNet:
            >>>     pass
            >>> backbones.register_module(ResNet)

        Args:
            name (str | None): The module name to be registered. If not
                specified, the class name will be used.
            force (bool, optional): Whether to override an existing class with
                the same name. Default: False.
            module (type): Module class to be registered.
        """
        if not isinstance(force, bool):
            raise TypeError(f'force must be a boolean, but got {type(force)}')
        # NOTE: This is a walkaround to be compatible with the old api,
        # while it may introduce unexpected bugs.
        if isinstance(name, type):
            return self.deprecated_register_module(name, force=force)

        # raise the error ahead of time
        if not (name is None or isinstance(name, str) or is_seq_of(name, str)):
            raise TypeError(
                'name must be either of None, an instance of str or a sequence'
                f'  of str, but got {type(name)}')

        # use it as a normal method: x.register_module(module=SomeClass)
        if module is not None:
            self._register_module(
                module_class=module, module_name=name, force=force)
            return module

        # use it as a decorator: @x.register_module()
        def _register(cls):
            self._register_module(
                module_class=cls, module_name=name, force=force)
            return cls

        return _register

In [3]:
import inspect
import warnings
from functools import partial
from collections import abc
def build_from_cfg(cfg, registry, default_args=None):
    """Build a module from config dict.

    Args:
        cfg (dict): Config dict. It should at least contain the key "type".
        registry (:obj:`Registry`): The registry to search the type from.
        default_args (dict, optional): Default initialization arguments.

    Returns:
        object: The constructed object.
    """
    if not isinstance(cfg, dict):
        raise TypeError(f'cfg must be a dict, but got {type(cfg)}')
    if 'type' not in cfg:
        if default_args is None or 'type' not in default_args:
            raise KeyError(
                '`cfg` or `default_args` must contain the key "type", '
                f'but got {cfg}\n{default_args}')
    if not isinstance(registry, Registry):
        raise TypeError('registry must be an mmcv.Registry object, '
                        f'but got {type(registry)}')
    if not (isinstance(default_args, dict) or default_args is None):
        raise TypeError('default_args must be a dict or None, '
                        f'but got {type(default_args)}')

    args = cfg.copy()

    if default_args is not None:
        for name, value in default_args.items():
            args.setdefault(name, value)

    obj_type = args.pop('type')
    if isinstance(obj_type, str):
        obj_cls = registry.get(obj_type)
        if obj_cls is None:
            raise KeyError(
                f'{obj_type} is not in the {registry.name} registry')
    elif inspect.isclass(obj_type):
        obj_cls = obj_type
    else:
        raise TypeError(
            f'type must be a str or valid type, but got {type(obj_type)}')
    try:
        return obj_cls(**args)
    except Exception as e:
        # Normal TypeError does not print class name.
        raise type(e)(f'{obj_cls.__name__}: {e}')

In [4]:
root_dir= r"D:\Video_matching"
data = dict(
    # Cấu hình cho tập dữ liệu huấn luyện
    train=dict(
        # Loại dataset sử dụng
        type="LabelFeatZipDataSet",
        # Đường dẫn đến file chứa ID của các video truy vấn
        vids_path=f"{root_dir}/data/meta/train/train_query_id.csv",  
        # Đường dẫn đến file zip chứa các đặc trưng của video
        feat_zip_path=f"{root_dir}/data/feat_zip/feats.zip", #TODO
        # Số khung hình tối đa trong một video
        max_frames=256,
        # Số lượng worker để tải dữ liệu
        num_workers=8,
        # Đường dẫn đến file chứa các truy vấn tích cực
        ann_vids_path=f"{root_dir}/data/meta/train/train_positive_query.txt"
    ),
    # Cấu hình cho tập dữ liệu đánh giá
    val=dict(
        # Loại dataset sử dụng
        type="LabelFeatZipDataSet",
        # Đường dẫn đến file chứa ID của các video truy vấn
        vids_path=f"{root_dir}/data/meta/val/val_query_id.csv",
        # Đường dẫn đến file zip chứa các đặc trưng của video
        feat_zip_path=f"{root_dir}/data/feat_zip/feats.zip", #TODO
        # Số khung hình tối đa trong một video
        max_frames=256,
        # Số lượng worker để tải dữ liệu
        num_workers=8,
        # Đường dẫn đến file chứa các truy vấn tích cực
        ann_vids_path=f"{root_dir}/data/meta/train/train_positive_query.txt"
        ) 
)    

In [5]:
# df = pd.read_csv(r"C:\Users\ORLab\Downloads\train_matching_ground_truth-Copy.csv")
# df['query_id'].unique()
# len(df['query_id'].unique())
# unique_query_ids = df['query_id'].unique()
# with open(f"{root_dir}/data/meta/train/train_positive_query_1.txt", 'w') as f:
#     for query_id in unique_query_ids:
#         f.write(f"{query_id}\n")

In [6]:
train_dataset = build_dataset(data['train'])

#### query vid num 248
#### query vid samples Q100555 Q100542 Q100442 Q100616 Q100689


In [7]:
val_dataset = build_dataset(data['val'])

#### query vid num 69
#### query vid samples Q100539 Q100588 Q100488 Q100550 Q100458


In [27]:
train_dataloader = DataLoader(
    dataset = train_dataset,
    batch_size = 8,
    num_workers = 0,
    shuffle = True,
    drop_last = True
)
val_dataloader = DataLoader(
    dataset = val_dataset,
    batch_size = 8,
    num_workers = 0,
    shuffle = False,
    drop_last = True
)

In [28]:
bert_path = r"C:\Users\ORLab\.cache\huggingface\models--roberta-base\snapshots\e2da8e2f811d1448a5b465c236feacd80ffbac7b"
config = AutoConfig.from_pretrained(bert_path)
bert = AutoModel.from_pretrained(bert_path, config=config)
feat_dim = 512
bert_dim = 768
max_frames = 256
# batch_size = 8
epochs = 10
import torch.nn as nn


Some weights of RobertaModel were not initialized from the model checkpoint at C:\Users\ORLab\.cache\huggingface\models--roberta-base\snapshots\e2da8e2f811d1448a5b465c236feacd80ffbac7b and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
# # Lớp MS: Mô hình Scoring để đánh giá video
# class MS(nn.Module):

#     def __init__(self): 
#         super(MS, self).__init__() # Kế thừa từ lớp cha nn.Module của PyTorch

#         # Lớp projection để chuyển đổi kích thước đặc trưng từ kích thước feat_dim đầu vào sang kích thước bert_dim để phù hợp với BERT
#         self.frame_proj = nn.Sequential(nn.Linear(feat_dim, bert_dim), nn.LayerNorm(bert_dim))

#         # Cấu hình và khởi tạo mô hình BERT
#         config = AutoConfig.from_pretrained(bert_path) # Tải cấu hình mô hình BERT từ đường dẫn args.bert_path
#         # config.gradient_checkpointing = args.gradient_checkpointing # Sử dụng gradient checkpointing nếu được cấu hình
#         self.bert = AutoModel.from_pretrained(bert_path, config=config) # Tải mô hình BERT từ đường dẫn args.bert_path
#         self.max_frames = max_frames # Số lượng frame tối đa

#         # Lớp projection cuối cùng để tạo ra điểm số
#         self.output_proj = nn.Linear(bert_dim * 2, 1) # Lớp Linear với đầu vào là bert_dim * 2 và đầu ra là 1

#     def forward(self, feats): #Đầu vào là features, đầu ra là logits(điểm số)
#         # Chuyển đổi đặc trưng video
#         print(f'Input shape: {feats.shape}')
#         vision_feats = self.frame_proj(feats)  # Chuyển đổi đặc trưng video từ kích thước feat_dim sang kích thước bert_dim
#         # Tạo mask cho các frame có giá trị
#         masks = feats.abs().sum(dim=2).gt(0) # Tạo mask cho các frame có giá trị, các frames có tổng giá trị tuyệt đối khác 0 sẽ được mask bằng True

#         # Thêm các token đặc biệt (CLS và SEP)
#         bz, device = vision_feats.size(0), vision_feats.device # Lấy kích thước batch size (= số lượng video trong batch) và thiết bị
#         text = torch.tensor([101, 102], dtype=torch.long)[None].to(device) # Tạo tensor text với giá trị [101, 102] và chuyển về thiết bị device
#         emb = self.bert.get_input_embeddings() # Lấy embedding đầu vào của các token từ mô hình BERT
#         text_emb = emb(text).expand((bz, -1, -1))  # Mở rộng embedding của các token CLS và SEP cho toàn bộ batch.
#         cls_emb, sep_emb = text_emb[:, 0], text_emb[:, 1] # Lưu trữ embedding của token CLS và SEP

#         '''
#         Đoạn mã này có mục đích tạo ra các token đặc biệt (CLS và SEP) để thêm vào chuỗi embedding của các frame video 
#         trước khi đưa chúng vào mô hình BERT.
#          Đây là cách thường được sử dụng khi làm việc với mô hình BERT để xử lý các chuỗi dữ liệu không phải văn bản, như video.
#         '''

#         # Kết hợp embedding của CLS, video frames, và SEP
#         inputs_embeds = torch.cat([cls_emb[:, None], vision_feats, sep_emb[:, None]], dim=1) # Kết hợp embedding của token CLS, các frame video, và SEP thành một tensor đầu vào cho BERT.
#         masks = torch.cat([torch.ones((bz, 2)).to(device), masks], dim=1) # Cập nhật mask bằng cách thêm các vị trí mask cho CLS và SEP (luôn bằng 1).

#         # Đưa qua mô hình BERT
#         states = self.bert(inputs_embeds=inputs_embeds, attention_mask=masks)[0] # Đưa chuỗi embedding  qua mô hình BERT để tạo ra các hidden states
#         # Tính trung bình các frame không bị mask
#         avg_pool = self._nonzero_avg_pool(states, masks)
#         cls_pool = states[:, 0] # Lấy embedding của token CLS
#         # Kết hợp embedding CLS và trung bình
#         cat_pool = torch.cat([cls_pool, avg_pool], dim=1)
#         # Tạo điểm số cuối cùng
#         logits = self.output_proj(cat_pool).squeeze(1)

#         return logits
    
    
#     def _nonzero_avg_pool(self,hidden, mask):
#         # Hàm tính trung bình các frame không bị mask
#         mask = mask.to(hidden.dtype)
#         hidden = hidden * mask[..., None]
#         length = mask.sum(dim=1, keepdim=True)
#         avg_pool = hidden.sum(dim=1) / (length + 1e-5)
#         return avg_pool

In [30]:
# MS()

In [31]:
import torch
import torch.nn as nn
from transformers import AutoConfig, AutoModel

class MS1(nn.Module):

    def __init__(self, feat_dim, bert_dim, bert_path, max_frames): 
        super(MS1, self).__init__() # Kế thừa từ lớp cha nn.Module của PyTorch

        # Lớp projection để chuyển đổi kích thước đặc trưng từ kích thước feat_dim đầu vào sang kích thước bert_dim để phù hợp với BERT
        self.frame_proj = nn.Sequential(nn.Linear(feat_dim, bert_dim), nn.LayerNorm(bert_dim))

        # Cấu hình và khởi tạo mô hình BERT
        config = AutoConfig.from_pretrained(bert_path) # Tải cấu hình mô hình BERT từ đường dẫn args.bert_path
        # config.gradient_checkpointing = args.gradient_checkpointing # Sử dụng gradient checkpointing nếu được cấu hình
        self.bert = AutoModel.from_pretrained(bert_path, config=config) # Tải mô hình BERT từ đường dẫn args.bert_path
        self.max_frames = max_frames # Số lượng frame tối đa

        # Lớp projection cuối cùng để tạo ra điểm số
        self.output_proj = nn.Linear(bert_dim * 2, 1) # Lớp Linear với đầu vào là bert_dim * 2 và đầu ra là 1

    def forward(self, feats): #Đầu vào là features, đầu ra là logits(điểm số)
        # Chuyển đổi đặc trưng video
        # print(f'Input shape: {feats.shape}')
        vision_feats = self.frame_proj(feats)  # Chuyển đổi đặc trưng video từ kích thước feat_dim sang kích thước bert_dim
        # print(f'Vision features shape: {vision_feats.shape}')
        
        # Tạo mask cho các frame có giá trị
        masks = feats.abs().sum(dim=2).gt(0) # Tạo mask cho các frame có giá trị, các frames có tổng giá trị tuyệt đối khác 0 sẽ được mask bằng True
        # print(f'Masks shape: {masks.shape}')

        # Thêm các token đặc biệt (CLS và SEP)
        bz, device = vision_feats.size(0), vision_feats.device # Lấy kích thước batch size (= số lượng video trong batch) và thiết bị
        text = torch.tensor([101, 102], dtype=torch.long)[None].to(device) # Tạo tensor text với giá trị [101, 102] và chuyển về thiết bị device
        emb = self.bert.get_input_embeddings() # Lấy embedding đầu vào của các token từ mô hình BERT
        text_emb = emb(text).expand((bz, -1, -1))  # Mở rộng embedding của các token CLS và SEP cho toàn bộ batch.
        cls_emb, sep_emb = text_emb[:, 0], text_emb[:, 1] # Lưu trữ embedding của token CLS và SEP
        # print(f'CLS embedding shape: {cls_emb.shape}')
        # print(f'SEP embedding shape: {sep_emb.shape}')

        # Kết hợp embedding của CLS, video frames, và SEP
        inputs_embeds = torch.cat([cls_emb[:, None], vision_feats, sep_emb[:, None]], dim=1) # Kết hợp embedding của token CLS, các frame video, và SEP thành một tensor đầu vào cho BERT.
        # print(f'Inputs embeds shape: {inputs_embeds.shape}')
        masks = torch.cat([torch.ones((bz, 2)).to(device), masks], dim=1) # Cập nhật mask bằng cách thêm các vị trí mask cho CLS và SEP (luôn bằng 1).
        # print(f'Updated masks shape: {masks.shape}')

        # Đưa qua mô hình BERT
        states = self.bert(inputs_embeds=inputs_embeds, attention_mask=masks)[0] # Đưa chuỗi embedding  qua mô hình BERT để tạo ra các hidden states
        # print(f'States shape: {states.shape}')
        
        # Tính trung bình các frame không bị mask
        avg_pool = self._nonzero_avg_pool(states, masks)
        # print(f'Average pool shape: {avg_pool.shape}')
        cls_pool = states[:, 0] # Lấy embedding của token CLS
        # print(f'CLS pool shape: {cls_pool.shape}')
        
        # Kết hợp embedding CLS và trung bình
        cat_pool = torch.cat([cls_pool, avg_pool], dim=1)
        # print(f'Concatenated pool shape: {cat_pool.shape}')
        
        # Tạo điểm số cuối cùng
        logits = self.output_proj(cat_pool).squeeze(1)
        # print(f'Logits shape: {logits.shape}')

        return logits
    
    def _nonzero_avg_pool(self, hidden, mask):
        # Hàm tính trung bình các frame không bị mask
        mask = mask.to(hidden.dtype)
        hidden = hidden * mask[..., None]
        length = mask.sum(dim=1, keepdim=True)
        avg_pool = hidden.sum(dim=1) / (length + 1e-5)
        return avg_pool

In [32]:
model = MS1(feat_dim, bert_dim, bert_path, max_frames)
model

Some weights of RobertaModel were not initialized from the model checkpoint at C:\Users\ORLab\.cache\huggingface\models--roberta-base\snapshots\e2da8e2f811d1448a5b465c236feacd80ffbac7b and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


MS1(
  (frame_proj): Sequential(
    (0): Linear(in_features=512, out_features=768, bias=True)
    (1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (bert): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
 

In [33]:
# h = model.forward(torch.randn(2, 256, 512))
# h

In [34]:
# model = MS()
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
model.to(device)

MS1(
  (frame_proj): Sequential(
    (0): Linear(in_features=512, out_features=768, bias=True)
    (1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (bert): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
 

In [35]:
warmup_ratio = 0.1
lr = 5e-5
batch_size = 8
opt = AdamW(model.parameters(), lr=lr)
stepsize = (len(train_dataset) // batch_size + 1)
total_steps = (len(train_dataset) // batch_size + 1) * epochs
scheduler = get_linear_schedule_with_warmup(opt, num_warmup_steps=warmup_ratio * total_steps,
                                            num_training_steps=total_steps)

d:\Miniconda\envs\myenv\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [36]:
# # Tải checkpoint nếu có
# start_epoch = 0
# ckpt = None
# if resume:
#     ckpt = torch.load(resume, map_location='cpu')
# elif os.path.exists(work_dir + '/last.txt'):
#     f = open(work_dir + '/last.txt')
#     e = int(f.readline())
#     f.close()
#     ckpt = torch.load(work_dir + '/checkpoints/epoch_%d.pth' % e, map_location='cpu')
# if ckpt is not None:
#     model.load_state_dict(ckpt['state_dict'])
#     opt.load_state_dict(ckpt['optimizer'])
#     scheduler.load_state_dict(ckpt['scheduler'])
#     start_epoch = ckpt['epoch'] + 1
#     del ckpt

In [37]:
# # Hàm thực hiện một bước huấn luyện
# def train_step(batch_data):
#     labels = batch_data["labels"]
#     frames = batch_data["frames"]
#     print(f'Frames shape: {frames.shape}')
#     # logits = model(frames)
#     # scores = logits.sigmoid()

#     # loss_ = F.binary_cross_entropy_with_logits(logits, labels.float())

#     # # scores, labels = all_gather(args.local_rank, world_size, scores=scores[:, None], labels=labels[:, None])
#     # # scores = scores.squeeze(1)
#     # # labels = labels.squeeze(1)

#     # acc = (scores.round() == labels).float().mean().item()
#     # pn = (labels == 1).sum() / labels.size(0)

#     # ap_ = average_precision_score(labels.detach().cpu().numpy(), scores.detach().cpu().numpy())

#     # return loss_, ap_, acc, pn
# # batch = train_dataloader[0]
# # train_step(batch)

In [38]:
labels = [1,0]

In [45]:
def train_step(batch_data):
    labels = batch_data["labels"].to(device)
    frames = batch_data["frames"].to(device)
    
    # In ra hình dáng của frames
    # print(f'Hình dáng của frames: {frames.shape}')
    
    # Tiếp tục các bước khác trong hàm train_step
    logits = model(frames)
    scores = logits.sigmoid()
    
    print("Scores:")
    print(scores)
    print('_____________________________')
    
    print("Labels:")
    print(labels)
    print('_____________________________')
    
    print("Rounded Scores:")
    print(scores.round())

    loss_ = F.binary_cross_entropy(logits, labels.float())

    acc = (scores.round() == labels).float().mean().item()
    pn = (labels == 1).sum() / labels.size(0)

    ap_ = average_precision_score(labels.detach().cpu().numpy(), scores.detach().cpu().numpy())

    return loss_, ap_, acc, pn

# Lấy một batch từ train_dataloader và gọi hàm train_step
for epoch in range(20):
    print(f'Epoch {epoch + 1 }')
    for batch in train_dataloader:
        loss_, ap_, acc, pn = train_step(batch)
        
        print("Loss:")
        print(loss_)
        print('____________')
        
        print("Average Precision (AP):")
        print(ap_)
        print('_____________________')
        
        print("Accuracy (Acc):")
        print(acc)
        print('________________')
        
        print("Positive Negative Ratio (PN):")
        print(pn)
        
#         break
# for batch in train_dataloader:
#     loss_, ap_, acc, pn = train_step(batch)
    
#     print("Loss:")
#     print(loss_)
#     print('____________')
    
#     print("Average Precision (AP):")
#     print(ap_)
#     print('_____________________')
    
#     print("Accuracy (Acc):")
#     print(acc)
#     print('________________')
    
#     print("Positive Negative Ratio (PN):")
#     print(pn)
    
#     break

Epoch 1
Scores:
tensor([0.5244, 0.5248, 0.5250, 0.5221, 0.5030, 0.5267, 0.5187, 0.5071],
       device='cuda:0', grad_fn=<SigmoidBackward0>)
_____________________________
Labels:
tensor([1, 0, 0, 0, 0, 0, 1, 0], device='cuda:0')
_____________________________
Rounded Scores:
tensor([1., 1., 1., 1., 1., 1., 1., 1.], device='cuda:0',
       grad_fn=<RoundBackward0>)
Loss:
tensor(0.6714, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
____________
Average Precision (AP):
0.29166666666666663
_____________________
Accuracy (Acc):
0.25
________________
Positive Negative Ratio (PN):
tensor(0.2500, device='cuda:0')
Scores:
tensor([0.5179, 0.5143, 0.5052, 0.5248, 0.5189, 0.5193, 0.5088, 0.5182],
       device='cuda:0', grad_fn=<SigmoidBackward0>)
_____________________________
Labels:
tensor([0, 1, 0, 1, 1, 0, 0, 0], device='cuda:0')
_____________________________
Rounded Scores:
tensor([1., 1., 1., 1., 1., 1., 1., 1.], device='cuda:0',
       grad_fn=<RoundBackward0>)
Loss:
tensor(1.0045,

d:\Miniconda\envs\myenv\Lib\site-packages\sklearn\metrics\_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
d:\Miniconda\envs\myenv\Lib\site-packages\sklearn\metrics\_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


tensor([0.5087, 0.5172, 0.5085, 0.5325, 0.5082, 0.5210, 0.5116, 0.5100],
       device='cuda:0', grad_fn=<SigmoidBackward0>)
_____________________________
Labels:
tensor([0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
_____________________________
Rounded Scores:
tensor([1., 1., 1., 1., 1., 1., 1., 1.], device='cuda:0',
       grad_fn=<RoundBackward0>)
Loss:
tensor(0.0613, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
____________
Average Precision (AP):
-0.0
_____________________
Accuracy (Acc):
0.0
________________
Positive Negative Ratio (PN):
tensor(0., device='cuda:0')
Scores:
tensor([0.5246, 0.5070, 0.5228, 0.5064, 0.5132, 0.5199, 0.5257, 0.5217],
       device='cuda:0', grad_fn=<SigmoidBackward0>)
_____________________________
Labels:
tensor([0, 0, 0, 0, 0, 0, 0, 1], device='cuda:0')
_____________________________
Rounded Scores:
tensor([1., 1., 1., 1., 1., 1., 1., 1.], device='cuda:0',
       grad_fn=<RoundBackward0>)
Loss:
tensor(0.3681, device='cuda:0', grad_fn=<BinaryCro

d:\Miniconda\envs\myenv\Lib\site-packages\sklearn\metrics\_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
d:\Miniconda\envs\myenv\Lib\site-packages\sklearn\metrics\_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


tensor([0.5132, 0.5244, 0.5044, 0.5192, 0.5229, 0.5081, 0.5122, 0.5113],
       device='cuda:0', grad_fn=<SigmoidBackward0>)
_____________________________
Labels:
tensor([0, 1, 0, 0, 0, 0, 0, 0], device='cuda:0')
_____________________________
Rounded Scores:
tensor([1., 1., 1., 1., 1., 1., 1., 1.], device='cuda:0',
       grad_fn=<RoundBackward0>)
Loss:
tensor(0.3381, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
____________
Average Precision (AP):
1.0
_____________________
Accuracy (Acc):
0.125
________________
Positive Negative Ratio (PN):
tensor(0.1250, device='cuda:0')
Scores:
tensor([0.5228, 0.5139, 0.5244, 0.5255, 0.5115, 0.5205, 0.5218, 0.5167],
       device='cuda:0', grad_fn=<SigmoidBackward0>)
_____________________________
Labels:
tensor([0, 0, 1, 0, 0, 0, 0, 1], device='cuda:0')
_____________________________
Rounded Scores:
tensor([1., 1., 1., 1., 1., 1., 1., 1.], device='cuda:0',
       grad_fn=<RoundBackward0>)
Loss:
tensor(0.6898, device='cuda:0', grad_fn=<Bina

...

d:\Miniconda\envs\myenv\Lib\site-packages\sklearn\metrics\_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


tensor([0.5241, 0.5040, 0.5121, 0.5071, 0.5058, 0.5244, 0.5146, 0.5187],
       device='cuda:0', grad_fn=<SigmoidBackward0>)
_____________________________
Labels:
tensor([1, 0, 0, 0, 0, 1, 0, 1], device='cuda:0')
_____________________________
Rounded Scores:
tensor([1., 1., 1., 1., 1., 1., 1., 1.], device='cuda:0',
       grad_fn=<RoundBackward0>)
Loss:
tensor(0.9288, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
____________
Average Precision (AP):
1.0
_____________________
Accuracy (Acc):
0.375
________________
Positive Negative Ratio (PN):
tensor(0.3750, device='cuda:0')
Scores:
tensor([0.5089, 0.5143, 0.5176, 0.5074, 0.5166, 0.5250, 0.5137, 0.5213],
       device='cuda:0', grad_fn=<SigmoidBackward0>)
_____________________________
Labels:
tensor([0, 0, 0, 0, 0, 0, 0, 1], device='cuda:0')
_____________________________
Rounded Scores:
tensor([1., 1., 1., 1., 1., 1., 1., 1.], device='cuda:0',
       grad_fn=<RoundBackward0>)
Loss:
tensor(0.3616, device='cuda:0', grad_fn=<Bina

d:\Miniconda\envs\myenv\Lib\site-packages\sklearn\metrics\_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
d:\Miniconda\envs\myenv\Lib\site-packages\sklearn\metrics\_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


tensor([0.5238, 0.5049, 0.5218, 0.5064, 0.5118, 0.5197, 0.5146, 0.5147],
       device='cuda:0', grad_fn=<SigmoidBackward0>)
_____________________________
Labels:
tensor([0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
_____________________________
Rounded Scores:
tensor([1., 1., 1., 1., 1., 1., 1., 1.], device='cuda:0',
       grad_fn=<RoundBackward0>)
Loss:
tensor(0.0610, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
____________
Average Precision (AP):
-0.0
_____________________
Accuracy (Acc):
0.0
________________
Positive Negative Ratio (PN):
tensor(0., device='cuda:0')
Scores:
tensor([0.5164, 0.5217, 0.5226, 0.5101, 0.5187, 0.5236, 0.5056, 0.5030],
       device='cuda:0', grad_fn=<SigmoidBackward0>)
_____________________________
Labels:
tensor([0, 1, 0, 0, 0, 0, 0, 0], device='cuda:0')
_____________________________
Rounded Scores:
tensor([1., 1., 1., 1., 1., 1., 1., 1.], device='cuda:0',
       grad_fn=<RoundBackward0>)
Loss:
tensor(0.3576, device='cuda:0', grad_fn=<BinaryCro

d:\Miniconda\envs\myenv\Lib\site-packages\sklearn\metrics\_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
d:\Miniconda\envs\myenv\Lib\site-packages\sklearn\metrics\_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


In [40]:
work_dir = r"D:\Video_matching"
# Tạo thư mục và logger cho 1 GPU
os.makedirs(work_dir, exist_ok=True)
os.makedirs(f"{work_dir}/checkpoints", exist_ok=True)

logger = logging.getLogger('log')
logger.setLevel(logging.INFO)

ch = logging.StreamHandler(stream=sys.stdout)
ch.setLevel(logging.INFO)
formatter = logging.Formatter("[%(levelname)s: %(asctime)s] %(message)s")
ch.setFormatter(formatter)
logger.addHandler(ch)

fh = logging.FileHandler(f"{work_dir}/log.txt")
fh.setLevel(logging.INFO)
fh.setFormatter(formatter)
logger.addHandler(fh)


In [41]:
# global_step = 0
# fp16 = True
# scaler = torch.cuda.amp.GradScaler()
# print_freq = 10
# eval_freq = 50
# # for e in range (epochs):
# #     model.train()
# #     # train_sampler.set_epoch(e)
# #     for b, batch in enumerate(train_dataloader):

# #         for _k, _v in batch.items():
# #             if isinstance(_v, torch.Tensor):
# #                 batch[_k] = _v.cuda()

# #         opt.zero_grad()
# #         if fp16:
# #             with torch.cuda.amp.autocast():
# #                 loss, ap, acc, pn = train_step(batch)
# #                 scaler.scale(loss).backward()
# #                 # torch.nn.utils.clip_grad_norm_(model.parameters(), args.clip_grad_norm)
# #                 scaler.step(opt)
# #                 scaler.update()
# #         else:
# #             loss, ap, acc, pn = train_step(batch)
# #             loss.backward()
# #             # torch.nn.utils.clip_grad_norm_(model.parameters(), args.clip_grad_norm)
# #             opt.step()
# #         scheduler.step()
# #         global_step += 1

# #         # if args.local_rank == 0 and b % print_freq == 0:
# #         #     logger.info('Epoch %d Batch %d Loss %.3f, AP %.3f, ACC %.3f, PN %.3f' % (e, b, loss.item(), ap, acc, pn))
# #         if b % print_freq == 0:
# #             logger.info('Epoch %d Batch %d Loss %.3f, AP %.3f, ACC %.3f, PN %.3f' % (e, b, loss.item(), ap, acc, pn))

#         if b > 0 and global_step % eval_freq == 0:
#             # Đánh giá mô hình
#             model.eval()
#             tot_val_labels = []
#             tot_val_scores = []
#             with torch.no_grad(), torch.cuda.amp.autocast():
#                 for val_batch in val_dataloader:
#                     for _k, _v in val_batch.items():
#                         if isinstance(_v, torch.Tensor):
#                             val_batch[_k] = _v.cuda()
#                     val_labels = val_batch["labels"]
#                     val_logits = model(val_batch["frames"].cuda())
#                     val_scores = val_logits.sigmoid()

#                     # val_scores, val_labels = all_gather(
#                     #     args.local_rank, world_size, val_scores=val_scores[:, None], val_labels=val_labels[:, None]
#                     # )
#                     val_scores = val_scores.squeeze()
#                     val_labels = val_labels.squeeze()

#                     tot_val_scores.append(val_scores.detach().cpu().numpy())
#                     tot_val_labels.append(val_labels.detach().cpu().numpy())

#             tot_val_labels = np.concatenate(tot_val_labels, axis=0)
#             tot_val_scores = np.concatenate(tot_val_scores, axis=0)

#             val_acc = (tot_val_scores.round() == tot_val_labels).mean()
#             val_ap = average_precision_score(tot_val_labels, tot_val_scores)

#             # if args.local_rank == 0:

#             print("*** Epoch %d Batch %d VAL AP %.3f, VAL ACC %.3f" % (e, b, val_ap, val_acc))
#             logger.info("*** Epoch %d Batch %d VAL AP %.3f, VAL ACC %.3f" % (e, b, val_ap, val_acc))
#             ckpt = {'state_dict': model.state_dict(), 'optimizer': opt.state_dict(), 'scheduler': scheduler.state_dict(),
#                     'epoch': e}
#             torch.save(ckpt, work_dir + '/checkpoints/epoch_%d_step_%d.pth' % (e, global_step))


In [48]:
for e in range(20): 
    print(f'Epoch {e+1}:')
    model.eval()
    tot_val_labels = []
    tot_val_scores = []
    with torch.no_grad() and torch.cuda.amp.autocast():

        for val_batch in val_dataloader:
            val_labels = val_batch['labels'].to(device)
            val_frames = val_batch['frames'].to(device)
            val_logits = model(val_frames).to(device)
            val_scores = val_logits.sigmoid()
            
            # In ra các giá trị của val_labels, val_frames, val_logits, val_scores
            # print("val_labels:")
            # print(val_labels)
            # print('_____________________________')
            
            # print("val_frames:")
            # print(val_frames)
            # print('_____________________________')
            
            # print("val_logits:")
            # print(val_logits)
            # print('_____________________________')
            
            # print("val_scores:")
            # print(val_scores)
            # print('_____________________________')
            
            tot_val_labels.extend(val_labels.detach().cpu().numpy())
            tot_val_scores.extend(val_scores.detach().cpu().numpy())
            
        
        # tot_val_labels = np.concatenate(tot_val_labels, axis=0)
        # tot_val_scores = np.concatenate(tot_val_scores, axis=0)
    # Chuyển đổi tot_val_labels và tot_val_scores thành tensor
    tot_val_labels = torch.tensor(tot_val_labels, dtype=torch.float32).to(device)
    tot_val_scores = torch.tensor(tot_val_scores, dtype=torch.float32).to(device)

    # In ra kích thước của tot_val_labels và tot_val_scores để kiểm tra
    print("Shape of tot_val_labels:")
    print(tot_val_labels.shape)
    print('_____________________________')

    print("Shape of tot_val_scores:")
    print(tot_val_scores.shape)
    print('_____________________________')

    # In ra các giá trị của tot_val_labels và tot_val_scores trước khi chuyển đổi thành tensor
    print("tot_val_labels before conversion to tensor:")
    print(tot_val_labels)
    print('_____________________________')

    print("tot_val_scores before conversion to tensor:")
    print(tot_val_scores)
    print('_____________________________')

    

    # In ra kích thước của tot_val_labels và tot_val_scores sau khi chuyển đổi thành tensor
    print("Shape of tot_val_labels after conversion to tensor:")
    print(tot_val_labels.shape)
    print('_____________________________')

    print("Shape of tot_val_scores after conversion to tensor:")
    print(tot_val_scores.shape)
    print('_____________________________')

    # In ra các giá trị của tot_val_labels và tot_val_scores sau khi chuyển đổi thành tensor
    print("tot_val_labels after conversion to tensor:")
    print(tot_val_labels)
    print('_____________________________')

    print("tot_val_scores after conversion to tensor:")
    print(tot_val_scores)
    print('_____________________________')
    print('Total_val_score rounded:')
    print(tot_val_scores.round())

    val_ap = average_precision_score(tot_val_labels.cpu().numpy(), tot_val_scores.cpu().numpy())
    val_acc = (tot_val_scores.round() == tot_val_labels).float().mean().item()
    # val_loss = F.binary_cross_entropy(tot_val_scores, tot_val_labels).float().item()

    # In ra các giá trị tính toán được
    print("Average Precision (AP):")
    print(val_ap)
    print('_____________________________')

    print("Accuracy (Acc):")
    print(val_acc)
    print('_____________________________')

    # print("Validation Loss:")
    # print(val_loss)

Epoch 1:


C:\Users\ORLab\AppData\Local\Temp\ipykernel_35468\112787271.py:6: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad() and torch.cuda.amp.autocast():


Shape of tot_val_labels:
torch.Size([64])
_____________________________
Shape of tot_val_scores:
torch.Size([64])
_____________________________
tot_val_labels before conversion to tensor:
tensor([0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 1., 1., 1.], device='cuda:0')
_____________________________
tot_val_scores before conversion to tensor:
tensor([0.5146, 0.5078, 0.5083, 0.5210, 0.5127, 0.5312, 0.5190, 0.5195, 0.5220,
        0.5107, 0.5151, 0.5264, 0.5122, 0.5210, 0.5205, 0.5054, 0.5190, 0.5288,
        0.5078, 0.5088, 0.5264, 0.5083, 0.5254, 0.5073, 0.5098, 0.5088, 0.5059,
        0.5093, 0.5200, 0.5146, 0.5166, 0.5127, 0.5093, 0.5269, 0.5034, 0.5127,
        0.5083, 0.5220, 0.5117, 0.5117, 0.5200, 0.5288, 0.5195, 0.5244, 0.5166,
        0.5229, 0.5225, 0.5293, 

C:\Users\ORLab\AppData\Local\Temp\ipykernel_35468\112787271.py:6: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad() and torch.cuda.amp.autocast():


Shape of tot_val_labels:
torch.Size([64])
_____________________________
Shape of tot_val_scores:
torch.Size([64])
_____________________________
tot_val_labels before conversion to tensor:
tensor([0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 1., 1., 1.], device='cuda:0')
_____________________________
tot_val_scores before conversion to tensor:
tensor([0.5146, 0.5078, 0.5083, 0.5210, 0.5127, 0.5312, 0.5190, 0.5195, 0.5220,
        0.5107, 0.5151, 0.5264, 0.5122, 0.5210, 0.5205, 0.5054, 0.5190, 0.5288,
        0.5078, 0.5088, 0.5264, 0.5083, 0.5254, 0.5073, 0.5098, 0.5088, 0.5059,
        0.5093, 0.5200, 0.5146, 0.5166, 0.5127, 0.5093, 0.5269, 0.5034, 0.5127,
        0.5083, 0.5220, 0.5117, 0.5117, 0.5200, 0.5288, 0.5195, 0.5244, 0.5166,
        0.5229, 0.5225, 0.5293, 

...

Shape of tot_val_labels:
torch.Size([64])
_____________________________
Shape of tot_val_scores:
torch.Size([64])
_____________________________
tot_val_labels before conversion to tensor:
tensor([0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 1., 1., 1.], device='cuda:0')
_____________________________
tot_val_scores before conversion to tensor:
tensor([0.5146, 0.5078, 0.5083, 0.5210, 0.5127, 0.5312, 0.5190, 0.5195, 0.5220,
        0.5107, 0.5151, 0.5264, 0.5122, 0.5210, 0.5205, 0.5054, 0.5190, 0.5288,
        0.5078, 0.5088, 0.5264, 0.5083, 0.5254, 0.5073, 0.5098, 0.5088, 0.5059,
        0.5093, 0.5200, 0.5146, 0.5166, 0.5127, 0.5093, 0.5269, 0.5034, 0.5127,
        0.5083, 0.5220, 0.5117, 0.5117, 0.5200, 0.5288, 0.5195, 0.5244, 0.5166,
        0.5229, 0.5225, 0.5293, 

C:\Users\ORLab\AppData\Local\Temp\ipykernel_35468\112787271.py:6: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad() and torch.cuda.amp.autocast():


Shape of tot_val_labels:
torch.Size([64])
_____________________________
Shape of tot_val_scores:
torch.Size([64])
_____________________________
tot_val_labels before conversion to tensor:
tensor([0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 1., 1., 1.], device='cuda:0')
_____________________________
tot_val_scores before conversion to tensor:
tensor([0.5146, 0.5078, 0.5083, 0.5210, 0.5127, 0.5312, 0.5190, 0.5195, 0.5220,
        0.5107, 0.5151, 0.5264, 0.5122, 0.5210, 0.5205, 0.5054, 0.5190, 0.5288,
        0.5078, 0.5088, 0.5264, 0.5083, 0.5254, 0.5073, 0.5098, 0.5088, 0.5059,
        0.5093, 0.5200, 0.5146, 0.5166, 0.5127, 0.5093, 0.5269, 0.5034, 0.5127,
        0.5083, 0.5220, 0.5117, 0.5117, 0.5200, 0.5288, 0.5195, 0.5244, 0.5166,
        0.5229, 0.5225, 0.5293, 

C:\Users\ORLab\AppData\Local\Temp\ipykernel_35468\112787271.py:6: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad() and torch.cuda.amp.autocast():


Shape of tot_val_labels:
torch.Size([64])
_____________________________
Shape of tot_val_scores:
torch.Size([64])
_____________________________
tot_val_labels before conversion to tensor:
tensor([0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 1., 1., 1.], device='cuda:0')
_____________________________
tot_val_scores before conversion to tensor:
tensor([0.5146, 0.5078, 0.5083, 0.5210, 0.5127, 0.5312, 0.5190, 0.5195, 0.5220,
        0.5107, 0.5151, 0.5264, 0.5122, 0.5210, 0.5205, 0.5054, 0.5190, 0.5288,
        0.5078, 0.5088, 0.5264, 0.5083, 0.5254, 0.5073, 0.5098, 0.5088, 0.5059,
        0.5093, 0.5200, 0.5146, 0.5166, 0.5127, 0.5093, 0.5269, 0.5034, 0.5127,
        0.5083, 0.5220, 0.5117, 0.5117, 0.5200, 0.5288, 0.5195, 0.5244, 0.5166,
        0.5229, 0.5225, 0.5293, 